In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
#from sklearn.feature_extraction.text import TfidfVectorizer
from DurationEmbedding import bin_cutoff, tfidf_embedding, pseudo_embedding_pipeline_two_phase, enhanced_auto_binning_pipeline, smart_auto_binning_pipeline

In [3]:
event = pd.read_csv("../input/Event_Feature_pro.csv")

In [4]:
event_embedding = event[['Case ID', 'Start Timestamp', 'Activity_code', 'duration']].copy()
#event_embedding = event[['case:concept:name', 'StartTime', 'pre_activity', 'Duration']].copy()
event_embedding.loc[:,'duration_min'] = np.ceil(event_embedding['duration']/60)

In [6]:
# this is for manu cut
event_embedding = bin_cutoff(event_embedding, 'duration_min', 5,24)

bin
0       9091
1        452
1011     168
1086     170
1166     174
1237     165
1296     169
1355     169
1419     170
1472     172
1536     168
1607     172
1678     165
1753     170
1834     169
1918     170
2        723
2032     169
2198     169
2578     170
3        454
4        134
5        170
5108     169
624      170
6359     170
793      169
917      170
Name: count, dtype: int64
Cut-off values for bins:
[5, 624)
[624, 793)
[793, 917)
[917, 1011)
[1011, 1086)
[1086, 1166)
[1166, 1237)
[1237, 1296)
[1296, 1355)
[1355, 1419)
[1419, 1472)
[1472, 1536)
[1536, 1607)
[1607, 1678)
[1678, 1753)
[1753, 1834)
[1834, 1918)
[1918, 2032)
[2032, 2198)
[2198, 2578)
[2578, 5108)
[5108, 6359)
[6359, 10742)


In [ ]:
#Semi-automatic (shows analysis)
event_embedding = pseudo_embedding_pipeline_two_phase(event_embedding, 'duration_min', optimize=True)

In [ ]:
#hard manu cut
event_embedding = pseudo_embedding_pipeline_two_phase(event_embedding, 'duration_min', t_cut=5, q_bins=23)

In [ ]:
# count for zero problem and distribution, completely auto -> prefer
event_embedding = enhanced_auto_binning_pipeline(event_embedding, 'duration_min')

In [ ]:
# Step 1: Let the system analyze and decide
#event_embedding, analysis = smart_auto_binning_pipeline(event_embedding, 'duration_min')
_, analysis = smart_auto_binning_pipeline(event_embedding, 'duration_min')

# See what it detected and why
print("Detected pattern:", analysis['strategy'])
print("Small value cutoff:", analysis['detected_small_cutoff']) 
print("Reasoning:", analysis['reasoning'])

In [10]:
#event_embedding = tfidf_embedding(event_embedding, "Case ID", "Activity_code", "bin")
event_embedding = tfidf_embedding(event_embedding, "case:concept:name", "pre_activity", "bin")

In [12]:
event_embedding.to_csv("../output/Event_Duration_Embedding_pro.csv", index = False)